In [71]:
!pip install pyshacl > /dev/null
!pip install rdflib > /dev/null


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


$$
\begin{table}
    \begin{tabular}{cc}
         Policy | Privacy & $p        ::=   p_{id} = [DUR]$\\
         DutyRule& $DUR ::= DUR_id = <A, t>$\\
         Action& $A ::= A_id = anonymize$\\
         Target& $t ::= t_id = target$\\
 
         & \\
    \end{tabular}
\end{table}
$$

In [72]:
from rdflib import *
tbox = Namespace('http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#')
abox = Namespace('http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#')
dcat = Namespace('https://www.w3.org/ns/dcat#')
dcterms = Namespace('http://purl.org/dc/terms/')
tb = Namespace("http://www.semanticweb.org/acraf/ontologies/2021/0/SDM#")
odrl = Namespace("http://www.w3.org/ns/odrl/2/")

## DEFINE GRAMMAR

In [73]:
contract = Graph()
contract.parse("/home/acraf/psr/tfm/Prototype/SideCar/FederateDataSource/contract.ttl", format="ttl")

<Graph identifier=N656bf5f5ce644c429efeb249055eaada (<class 'rdflib.graph.Graph'>)>

### Shapes Graph

In [74]:
shapes = Graph()        
shapes.bind("odrl", "http://www.w3.org/ns/odrl/2/")
shapes.bind("tb", "http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#")
shapes.bind("ab", "http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#")
shapes = shapes.parse("./policy_grammar.ttl", format="turtle")
print(shapes.serialize(format="turtle"))

@prefix odrl: <http://www.w3.org/ns/odrl/2/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix tb: <http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

tb:PrivacyShape a sh:NodeShape ;
    sh:property [ sh:minCount 1 ;
            sh:node tb:DutyShape ;
            sh:path odrl:duty ] ;
    sh:targetClass odrl:Privacy .

tb:ActionShape a sh:NodeShape ;
    sh:property [ sh:hasValue odrl:Action ;
            sh:path rdf:type ] ;
    sh:targetClass odrl:anonymize .

tb:DutyShape a sh:NodeShape ;
    sh:property [ sh:minCount 1 ;
            sh:nodeKind sh:IRI ;
            sh:path odrl:target ],
        [ sh:minCount 1 ;
            sh:node tb:ActionShape ;
            sh:path odrl:action ] ;
    sh:targetClass odrl:Duty .




In [75]:
print(contract.serialize(format="turtle"))

@prefix ab: <http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix odrl: <http://www.w3.org/ns/odrl/2/> .
@prefix tb: <http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#> .

ab:UPENN-GBM_clinical_info_v2 a <https://www.w3.org/ns/dcat#Dataset> ;
    dcterms:identifier "b4f5450e04e5c1669e36df0e37e5e172872e9cce5924c24955222d7f58004afc" ;
    tb:hasAttribute ab:Age_at_scan_years,
        ab:GTR_over90percent,
        ab:Gender,
        ab:ID,
        ab:IDH1,
        ab:KPS,
        ab:MGMT,
        ab:PsP_TP_score,
        ab:Survival_Censor,
        ab:Survival_Status,
        ab:Survival_from_surgery_days_UPDATED,
        ab:Time_since_baseline_preop ;
    tb:hasDC ab:dc_UPENN-GBM_clinical_info_v2 ;
    tb:hasDTT ab:Tabular ;
    tb:hasTA ab:TA1 ;
    tb:owner "UPENN" .

ab:Age_at_scan_years a tb:Attribute ;
    tb:attribute "Age_at_scan_years" .

ab:AnonDuty a odrl:Duty ;
    odrl:action odrl:anonymize

### SHACL Validation

In [76]:
from pyshacl import validate

In [77]:
# Perform SHACL validation
conforms, report_graph, report_text = validate(contract,shacl_graph=shapes)

# Output the result
print(f"Report Text:\n{report_text}")


Report Text:
Validation Report
Conforms: True

